In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DESCRIPTION

Resample CMIP6 to study grid.

"""

# Import modules
import numpy as np
import pyresample
import glob
import netCDF4
import matplotlib.pyplot as plt
import pathlib

# Define path name
path = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/'

# Define savepath
sp = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer-resampled/'

# Get a list of all models
models = sorted(glob.glob(path + 'cmip6-summer/*/*.nc'))

In [8]:
# Define ice sheet grid
ismip = netCDF4.Dataset(path + 'masks/1km-ISMIP6.nc')
ismip_lon = ismip.variables['lon'][:]
ismip_lat = ismip.variables['lat'][:]
ismip_mask = ismip.variables['ICE'][:]

for i in models:
        
    # Get some file names
    p = pathlib.Path(i)
    folder = p.parent.name
    savepath = sp + folder + '/' + p.name
    savepath_path = pathlib.Path(sp + folder + '/' + p.name)
    
    if savepath_path.is_file():
        print('Skipping %s' %p)
    
    else:

        # Read CMIP6 data
        cmip = netCDF4.Dataset(i)

        # Get lat and lon
        cmip_lon_1d, cmip_lat_1d = cmip.variables['longitude'][:], cmip.variables['latitude'][:]

        # Convert to 2D array
        cmip_lon, cmip_lat = np.meshgrid(cmip_lon_1d, cmip_lat_1d)

        # Define grid using pyresample       
        swath_def = pyresample.geometry.GridDefinition(lons=ismip_lon, lats=ismip_lat)
        swath_con = pyresample.geometry.GridDefinition(lons=cmip_lon, lats=cmip_lat)

        # Determine nearest (w.r.t. great circle distance) neighbour in the grid.
        region_swath = pyresample.kd_tree.resample_nearest(source_geo_def=swath_con, 
                                                     target_geo_def=swath_def, 
                                                     data=cmip['t2m'][:], 
                                                     radius_of_influence=100000)

        ###############################################################################
        # Save 1 km dataset to NetCDF
        ###############################################################################
        dataset = netCDF4.Dataset(savepath, 'w', format='NETCDF4_CLASSIC')
        print('Creating... %s' % savepath)
        dataset.Title = "Mean summer 2 m air temperature"
        import time
        dataset.History = "Created " + time.ctime(time.time())
        dataset.Projection = "WGS 84"
        dataset.Reference = "Ryan, J. C., Smith. L. C., Cooley, S. W., and Pearson, B. (in review), Emerging importance of clouds for Greenland Ice Sheet energy balance and meltwater production."
        dataset.Contact = "jryan4@uoregon.edu"

        # Create new dimensions
        lat_dim = dataset.createDimension('y', region_swath.shape[0])
        lon_dim = dataset.createDimension('x', region_swath.shape[1])
        data_dim = dataset.createDimension('z', region_swath.shape[2])

        # Define variable types
        Y = dataset.createVariable('latitude', np.float32, ('y', 'x'))
        X = dataset.createVariable('longitude', np.float32, ('y', 'x'))

        # Define units
        Y.units = "degrees"
        X.units = "degrees"

        # Create the actual 3D variable
        t2m_nc = dataset.createVariable('t2m', np.float32, ('y','x','z'))

        # Write data to layers
        Y[:] = ismip_lat
        X[:] = ismip_lon
        t2m_nc[:] = region_swath

        print('Writing data to %s' % savepath)

        # Close dataset
        dataset.close()

Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/CMIP6_ACCESS-CM2_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/CMIP6_ACCESS-ESM1-5_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/CMIP6_CESM2-WACCM_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/CMIP6_CESM2_Amon_ssp126_r4i1p1f1_tas_2015-2100.nc
Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/CMIP6_CMCC-CM2-SR5_Amon_ssp126_r1i1p1f1_tas_2015-2100.nc
Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/CMIP6_CNRM-CM6-1-HR_Amon_ssp126_r1i1p1f2_tas_2015-2100.nc
Skipping /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/cmip6-summer/SSP126/C